## Rebuild Instances

1. Install novaclient & enter password when prompted:

```
yum -y install python-devel python-pip
pip install python-novaclient

#export OS_TENANT_ID=5c602a1d6f504e0f9346c68b5fefef71
#export OS_TENANT_NAME="permanent"
#export OS_USERNAME="pregion"

export OS_TENANT_ID=89042cdec81c43d39df4a244b4a487c9
export OS_TENANT_NAME="secloud"
export OS_USERNAME="sroberts"

export OS_AUTH_URL=http://172.24.74.252:5000/v2.0
echo "Please enter your OpenStack Password: "
read -sr OS_PASSWORD_INPUT
```

2. Set OS_PASSWORD
```
export OS_PASSWORD=$OS_PASSWORD_INPUT
```

3. Rebuild the instances. If running form one of these, make sure it is the last:

```
nova rebuild sroberts-bp02 sroberts-ambari2-server
nova rebuild sroberts-bp04 sroberts-ambari2-agent
nova rebuild sroberts-bp05 sroberts-ambari2-agent
nova rebuild sroberts-bp06 sroberts-ambari2-agent
nova rebuild sroberts-bp07 sroberts-ambari2-agent
nova rebuild sroberts-bp08 sroberts-ambari2-agent

```

4. Or build new instances:

```
nova boot --flavor m3.large --image sroberts-ambari2-agent sroberts-bp03
```

## Preparing Nodes

#### Deploy Ambari Server & Agent

```bash
stack=sroberts-bp
ambari_server=${stack}02.cloud.hortonworks.com
bootstrap_url=https://raw.githubusercontent.com/seanorama/ambari-bootstrap/deploy-ambari2/ambari-bootstrap.sh
 
## link /hadoop to /mnt to use the data disk
pdsh -w ${stack}0[1-9].cloud.hortonworks.com "ln -sf /mnt /hadoop; exit"
 
## install the ambari-server
pdsh -w ${ambari_server} "curl -sSL ${bootstrap_url} | install_ambari_server=true sh"

## install to all other nodes. See ‘man pdsh’ for the various ways to specify hosts.
pdsh -w ${stack}0[2-9].cloud.hortonworks.com "curl -sSL ${bootstrap_url} | sh; exit"
```


#### Raise Ambari Timeouts

```bash
pdsh -w ${ambari_server} \
  "sed -e 's/\(agent.task.timeout=\).*/\12700/' \
       -e 's/\(agent.package.install.task.timeout=\).*/\12700/' \
       -i /etc/ambari-server/conf/ambari.properties"
       
pdsh -w ${ambari_server} "service ambari-server restart"
```

#### Use the local repo

```bash
for repo in HDP-2.2 HDP-UTILS-1.1.0.20; do
    curl -sSu admin http://${ambari_server}:8080/api/v1/stacks/HDP/versions/2.2/operating_systems/redhat6/repositories/${repo} -o /tmp/update-repo.txt
    sed -ir -e 's/\(public\|private\)-repo-1.hortonworks.com/repo.cloud.hortonworks.com/g' -e '/^  "href"/d' /tmp/update-repo.txt
    curl -sSu admin -H x-requested-by:sean http://${ambari_server}:8080/api/v1/stacks/HDP/versions/2.2/operating_systems/redhat6/repositories/${repo} -T /tmp/update-repo.txt
done
```

#### Node Registration

```bash
pdsh -w ${stack}0[3-8].cloud.hortonworks.com "sed -i 's/hostname=.*/hostname=sroberts-bp02.cloud.hortonworks.com/' /etc/ambari-agent/conf/ambari-agent.ini; service ambari-agent restart"
```